In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import mysql.connector


In [3]:
np.random.seed(500)

In [4]:
connection = mysql.connector.connect(
        host='ds50-mysql-do-user-9644544-0.b.db.ondigitalocean.com',
        database='ds50',
        user='ds50',
        password='AVNS_4ybSd0CoPKnCL5F',
        port = '25060')
if connection.is_connected():
        data = pd.read_sql( 'SELECT * FROM REVIEW LIMIT 5000', connection)
        connection.close()
data=pd.DataFrame(data)


C:\Users\antoi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
data = data[(data['review_text'].str.len() > 3) & (data['review_text'].str.len() < 3000) & (((~ data['review_text'].str.isdigit()) & (data['review_text'].str.len() != 0)) | (data['rating'] != 0))]


In [6]:
data.shape
data = data.reset_index()
data = data.drop(columns=["index"])
data.head(15)

,review_id,user_id,book_id,rating,review_text,date_added,started_at,n_votes,n_comments
0,00005dd943695f048c7348a06c6408db,296741,9409458,2,The final book in the Mercy Falls trilogy left...,2017-08-23 13:16:33,2017-08-23 07:00:00,0,0
1,0000f2fa08114079d9d7215bc5e754d2,155648,24817626,1,I reread To Kill a Mockingbird prior to readin...,2015-04-25 11:36:35,2016-08-13 07:00:00,1,0
2,0001a480a5b24fdad0633417a76b6bb0,266353,26891429,5,A very satisfying conclusion to this entertain...,2016-06-18 21:15:55,2016-06-18 07:00:00,0,0
3,0001b032e98876009c59acc94cca8194,91247,17167166,5,"""Why are you crying?"" \n ""Because,"" she whispe...",2015-12-18 15:34:29,2016-09-29 07:00:00,29,0
4,00020adeea578404ffa05e295fb2b182,253237,18966322,0,"I am interested in this world, but the series ...",2014-01-10 07:41:32,NaT,0,0
5,000226ba8be15e77f5dc8bd833c4b35e,29129,3257136,4,Dewey: The Small-Town Library Cat Who Touched ...,2008-12-29 22:47:59,NaT,1,0
6,000290a64894dd0368ae64ea3999b618,188877,18798983,4,A reinvention of the 'Thousand Arabian Nights'...,2016-02-16 19:18:39,NaT,0,0
7,0002d163004e23746d389390b3c39975,246898,12607835,5,Becoming a Butterfly discusses a topic that I'...,2012-04-25 22:26:30,2012-04-25 07:00:00,2,0
8,0002d890f8cb3236ff342f07e59ffd0b,17369,6514,4,** spoiler alert ** \n I was reading so many b...,2009-02-14 05:46:35,NaT,0,0
9,0002e65d17b9eba2098951b225ff28ae,236576,16101128,4,The 5th Wave is a post apocalyptic novel deali...,2013-05-18 15:29:11,2013-05-18 07:00:00,1,0


In [7]:
#On supprime les ligne de commentaire vide
data['review_text'].dropna(inplace=True)

#On met tout en minuscule
data['review_text'] = [entry.lower() for entry in data['review_text']]

data['review_text'] = [word_tokenize(entry) for entry in data['review_text']]

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(data['review_text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'review_text'] = str(Final_words)

In [8]:

data[data['rating'] == 0].head()

,review_id,user_id,book_id,rating,review_text,date_added,started_at,n_votes,n_comments
4,00020adeea578404ffa05e295fb2b182,253237,18966322,0,"['interested', 'world', 'series', 'ya', 'sligh...",2014-01-10 07:41:32,NaT,0,0
44,0007d6eff54a2588045d2b51a29a59bd,126657,9659607,0,"['iron', 'fey', 'series', 'definitely', 'one',...",2011-10-09 03:27:18,2011-10-08 07:00:00,1,0
56,00099c8e53d5f8d41aad970d964179a5,60325,2536134,0,"['niece', 'pick', 'library', 'pull', 'goodread...",2015-09-05 22:20:08,NaT,1,1
88,000f20f9182eee32cfb0f58dd9dce4f2,35934,19540434,0,"['excited', 'celebrate', 'release', 'unforgive...",2014-09-20 19:39:15,NaT,0,0
92,0010089a92b151283e08ba8699ef6469,39985,11990677,0,"['entertaining', 'expect', 'great', 'writing',...",2012-03-04 19:14:07,2012-03-04 08:00:00,0,0


**Ratings count**

In [15]:
#création du jeu de donnée sans les zéros

datasans0 = data[data['rating'] != 0]

In [16]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(datasans0['review_text'],datasans0['rating'],test_size=0.3)

In [24]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(type(Test_X))

<class 'pandas.core.series.Series'>


In [18]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['review_text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [19]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  45.25495750708215


On voit que la précision est de 45% ce qui est plutôt bon !!

In [29]:
Test_X_Tfidf = Tfidf_vect.transform(data[data['rating'] == 0]['review_text'])
predictions_sentiment_SVM = SVM.predict(Test_X_Tfidf)

k = 0

for i in range(len(data)):
    if data['rating'] == 0

C:\Users\antoi\AppData\Local\Temp\ipykernel_11716\2490492881.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['rating'] == 0]['rating'] = predictions_sentiment_SVM


In [30]:
data[data['rating'] == 0]

,review_id,user_id,book_id,rating,review_text,date_added,started_at,n_votes,n_comments
4,00020adeea578404ffa05e295fb2b182,253237,18966322,0,"['interested', 'world', 'series', 'ya', 'sligh...",2014-01-10 07:41:32,NaT,0,0
44,0007d6eff54a2588045d2b51a29a59bd,126657,9659607,0,"['iron', 'fey', 'series', 'definitely', 'one',...",2011-10-09 03:27:18,2011-10-08 07:00:00,1,0
56,00099c8e53d5f8d41aad970d964179a5,60325,2536134,0,"['niece', 'pick', 'library', 'pull', 'goodread...",2015-09-05 22:20:08,NaT,1,1
88,000f20f9182eee32cfb0f58dd9dce4f2,35934,19540434,0,"['excited', 'celebrate', 'release', 'unforgive...",2014-09-20 19:39:15,NaT,0,0
92,0010089a92b151283e08ba8699ef6469,39985,11990677,0,"['entertaining', 'expect', 'great', 'writing',...",2012-03-04 19:14:07,2012-03-04 08:00:00,0,0
...,...,...,...,...,...,...,...,...,...
4665,0287b072c261dff2dffd26bf5dd8c06c,410824,15828366,0,"['take', 'awhile', 'read', 'write', 'review', ...",2013-11-03 21:55:25,NaT,2,0
4668,0287fddf04ab2a54d17ad6623f29b3f4,256339,6310,0,"['take', 'proably', 'year', 'father', 'read', ...",2010-08-10 16:37:27,NaT,0,0
4685,028a76ea0255ca2cfe704031094c299a,95429,22443261,0,"['feel', 'really', 'bad', 'dnfing', 'brandon',...",2014-12-27 08:46:31,NaT,1,0
4731,028faeb5197cb79eb588dde77a20ae2f,350531,10830444,0,"['like', 'dont', 'like', 'prison', 'story', 'p...",2012-04-07 21:37:52,2012-04-07 07:00:00,1,1
